In [12]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# 


# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#Added 23.05.2018 at 13:55
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import keras
from keras.models import Sequential
from keras.layers import Dense

In [13]:
from keras.callbacks import TensorBoard
tbCallback=keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [14]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [15]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

In [16]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [17]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [18]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
#import torch
#import torch.nn as nn
#from torch.autograd import Variable
#import torch.nn.functional as F
#from torch import optim

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


from keras import backend as K
from keras import layers
import numpy as np
import tensorflow as tf

class PlasticDense(layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        #self.batch_size = batch_size
        super(PlasticDense, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        #Define set of traditional weights
        self.w = self.add_weight(name='w', 
                                 shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        
        #Define our plasticity coefficient
        self.alpha = self.add_weight(name='alpha', 
                              shape=(1, 1),
                              initializer='uniform',
                              trainable=True)
        
        
        #The Hebbian trace
        self.hebb = self.add_weight(name='hebb', 
                              shape=(input_shape[1], self.output_dim),
                              initializer='zeros',
                              trainable=False)
        
        #Step size will be optimized
        self.eta = self.add_weight(name='eta', 
                                      shape=(1, 1),
                                      initializer='uniform',
                                      trainable=False)
        super(PlasticDense, self).build(input_shape)

        
        #yout = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + input )
        #hebb = (1 - self.eta) * hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0] # bmm here is used to implement an outer product between yin and yout, with the help of unsqueeze (i.e. added empty dimensions)
        #return yout, hebb

    def call(self, x):
        
        #X (layer input)     : shape(?, INPUT_DIM)
        #W                   : shape(INPUT_DIM, OUTPUT_DIM)
        #hebb                : shape(INPUT_DIM, OUTPUT_DIM)
        #Y (layer output)    : shape(?, OUTPUT_DIM)
        #ETA                 : scalar (one per layer)
        
        #yout = K.maximum(0.0, np.add((K.dot(self.y, np.add(K.dot(self.alpha, K.transpose(self.hebb)), self.w))), x))
        #hebb = (1 - 0.01) * self.hebb + 0.01 * K.dot(self.y, yout)
        #yout = K.maximum(0.0, np.add(self.y * np.add(self.alpha * self.hebb, self.w), x))
        
        #y = K.dot(x, self.w)
        #y = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + yin )
        #plastic_y = self.alpha * (K.dot(x, self.hebb)) 
        #self.hebb = (1 - self.eta) * self.hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0]
        #model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        #self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        #print(self.hebb)
        
        #Hebbian update - option 2
        #self.hebb +=self.eta * K.dot(model_out, (x - (K.dot(model_out, self.hebb))))
        
        y = K.dot(x, self.w)
        plastic_y = self.alpha * (K.dot(x, self.hebb))   
        model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        print(self.hebb)

        return model_out
        #return K.maximum(0.0, y)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


In [20]:
def testClassic(num_neurons, num_layers, epochs):
    print("Classic: n: {} l: {} e: {}".format(num_neurons, num_layers, epochs))
    classifier = Sequential()
    classifier.add(Dense(output_dim = num_neurons, init = 'uniform', activation = 'relu', input_dim = 11))
    for l in range(num_layers - 1):
        classifier.add(Dense(output_dim = num_neurons, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    train_history = classifier.fit(X_train, y_train, batch_size = 128, nb_epoch = epochs)
    
    loss_history = train_history.history["loss"]
    np.savetxt("results/n_{}_l_{}_e_{}_loss_classic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(loss_history), delimiter=",")
    
    acc_history = train_history.history["acc"]
    np.savetxt("results/n_{}_l_{}_e_{}_acc_classic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(acc_history), delimiter=",")
    # TODO Append test prediction results to file or to its own file
    y_pred = classifier.predict(X_test)
    return (y_pred > 0.5)


def testPlastic(num_neurons, num_layers, epochs):
    print("Plastic: n: {} l: {} e: {}".format(num_neurons, num_layers, epochs))
    classifier = Sequential()
    classifier.add(PlasticDense(num_neurons, input_shape=(11,)))
    for l in range(num_layers - 1):
        classifier.add(PlasticDense(num_neurons))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    #Added 23.05.2018 at 13:55
    train_history = classifier.fit(X_train, y_train, batch_size = 128, nb_epoch = epochs, validation_split=0.2,callbacks=[tbCallback])
    #train_history = classifier.fit(X_train, y_train, batch_size = 128, nb_epoch = epochs)
        
    loss_history = train_history.history["loss"]
    np.savetxt("results/n_{}_l_{}_e_{}_loss_plastic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(loss_history), delimiter=",")
    
    acc_history = train_history.history["acc"]
    np.savetxt("results/n_{}_l_{}_e_{}_acc_plastic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(acc_history), delimiter=",")
    # TODO Append test prediction results to file or to its own file
    y_pred = classifier.predict(X_test)
    return (y_pred > 0.5)
    
    
def extensive_test():
    neuron_combos = [4, 6, 8, 12, 16]
    layer_combos = [2, 4, 8, 16]
    epochs = 200
    
    i = 0
    for n in neuron_combos:
        for l in layer_combos:
            testClassic(n, l, epochs)
            testPlastic(n, l, epochs)
            i += 1
            print("Try: {}/{}".format(i, len(neuron_combos)*len(layer_combos)))

extensive_test()
#y_pred = testClassic(4, 4, 2)
#y_pred = testPlastic(4, 4, 20)

Classic: n: 4 l: 2 e: 200


/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=4, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=4, kernel_initializer="uniform")`
  
/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  import sys
/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/200
8000/8000 [==============================] - 1s 66us/step - loss: 0.6827 - acc: 0.7864
Epoch 2/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.6463 - acc: 0.7960
Epoch 3/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.5710 - acc: 0.7960
Epoch 4/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4944 - acc: 0.7960
Epoch 5/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4579 - acc: 0.7960
Epoch 6/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4450 - acc: 0.7960
Epoch 7/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4398 - acc: 0.7960
Epoch 8/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4369 - acc: 0.7960
Epoch 9/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4350 - acc: 0.7960
Epoch 10/200
8000/8000 [==============================] - 0s 10us/step - loss: 0.4338 - acc: 0.7960
Epoch 11/

8000/8000 [==============================] - 0s 11us/step - loss: 0.4067 - acc: 0.8330
Epoch 84/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4066 - acc: 0.8335
Epoch 85/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4063 - acc: 0.8334
Epoch 86/200
8000/8000 [==============================] - 0s 10us/step - loss: 0.4063 - acc: 0.8331
Epoch 87/200
8000/8000 [==============================] - 0s 10us/step - loss: 0.4062 - acc: 0.8331
Epoch 88/200
8000/8000 [==============================] - 0s 10us/step - loss: 0.4061 - acc: 0.8337
Epoch 89/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4059 - acc: 0.8339
Epoch 90/200
8000/8000 [==============================] - 0s 10us/step - loss: 0.4060 - acc: 0.8329
Epoch 91/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4059 - acc: 0.8344
Epoch 92/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4058 - acc: 0.8339
Epoch 93/200


8000/8000 [==============================] - 0s 11us/step - loss: 0.4024 - acc: 0.8349
Epoch 165/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4025 - acc: 0.8354
Epoch 166/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4025 - acc: 0.8354
Epoch 167/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4023 - acc: 0.8361
Epoch 168/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4023 - acc: 0.8357
Epoch 169/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4023 - acc: 0.8342
Epoch 170/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4021 - acc: 0.8354
Epoch 171/200
8000/8000 [==============================] - 0s 10us/step - loss: 0.4021 - acc: 0.8347
Epoch 172/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4021 - acc: 0.8361
Epoch 173/200
8000/8000 [==============================] - 0s 11us/step - loss: 0.4021 - acc: 0.8350
Epoc

...

Epoch 42/200
6400/6400 [==============================] - 0s 24us/step - loss: 0.5080 - acc: 0.7958 - val_loss: 0.5065 - val_acc: 0.7969
Epoch 43/200
6400/6400 [==============================] - 0s 27us/step - loss: 0.5077 - acc: 0.7958 - val_loss: 0.5062 - val_acc: 0.7969
Epoch 44/200
6400/6400 [==============================] - 0s 24us/step - loss: 0.5075 - acc: 0.7958 - val_loss: 0.5060 - val_acc: 0.7969
Epoch 45/200
6400/6400 [==============================] - 0s 23us/step - loss: 0.5073 - acc: 0.7958 - val_loss: 0.5058 - val_acc: 0.7969
Epoch 46/200
6400/6400 [==============================] - 0s 23us/step - loss: 0.5071 - acc: 0.7958 - val_loss: 0.5057 - val_acc: 0.7969
Epoch 47/200
6400/6400 [==============================] - 0s 23us/step - loss: 0.5070 - acc: 0.7958 - val_loss: 0.5055 - val_acc: 0.7969
Epoch 48/200
6400/6400 [==============================] - 0s 23us/step - loss: 0.5068 - acc: 0.7958 - val_loss: 0.5054 - val_acc: 0.7969
Epoch 49/200
6400/6400 [=================

Epoch 102/200
6400/6400 [==============================] - 0s 25us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 103/200
6400/6400 [==============================] - 0s 24us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 104/200
6400/6400 [==============================] - 0s 24us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 105/200
6400/6400 [==============================] - 0s 24us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 106/200
6400/6400 [==============================] - 0s 23us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 107/200
6400/6400 [==============================] - 0s 23us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 108/200
6400/6400 [==============================] - 0s 27us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 109/200
6400/6400 [=========

6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 162/200
6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 163/200
6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 164/200
6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 165/200
6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 166/200
6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 167/200
6400/6400 [==============================] - 0s 21us/step - loss: 0.5062 - acc: 0.7958 - val_loss: 0.5047 - val_acc: 0.7969
Epoch 168/200
6400/6400 [=======================

8000/8000 [==============================] - 0s 24us/step - loss: 0.5061 - acc: 0.7960
Epoch 28/200
8000/8000 [==============================] - 0s 22us/step - loss: 0.5061 - acc: 0.7960
Epoch 29/200
8000/8000 [==============================] - 0s 22us/step - loss: 0.5061 - acc: 0.7960
Epoch 30/200
8000/8000 [==============================] - 0s 30us/step - loss: 0.5060 - acc: 0.7960
Epoch 31/200
8000/8000 [==============================] - 0s 25us/step - loss: 0.5062 - acc: 0.7960
Epoch 32/200
8000/8000 [==============================] - 0s 25us/step - loss: 0.5062 - acc: 0.7960
Epoch 33/200
8000/8000 [==============================] - 0s 25us/step - loss: 0.5061 - acc: 0.7960
Epoch 34/200
8000/8000 [==============================] - 0s 25us/step - loss: 0.5060 - acc: 0.7960
Epoch 35/200
8000/8000 [==============================] - 0s 25us/step - loss: 0.5064 - acc: 0.7960
Epoch 36/200
8000/8000 [==============================] - 0s 24us/step - loss: 0.5061 - acc: 0.7960
Epoch 37/200


KeyboardInterrupt: 

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)